## Time series
Plik w którym tworze jeden odpowiedni szereg czasowy

In [209]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [210]:
analyzer = SentimentIntensityAnalyzer()

In [211]:
# analiza sentymentu vader (najpierw sam)
# podzielic tweety po firmie [done]
# aggregacja godzinowo po max, min, mean, median, std, count sentymentu
# 

In [212]:
tweets = pd.read_csv('company_tweets/apple_tweets.csv')
#tweets = tweets.sample(10_000).reset_index(drop=True)

In [213]:
tweets.head()

,tweet_id,post_date,body,comment_num,retweet_num,like_num,ticker_symbol,company_name
0,550441509175443456,2015-01-01 01:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,AAPL,apple
1,550441672312512512,2015-01-01 01:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0,AAPL,apple
2,550443807834402816,2015-01-01 01:10:05,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,apple
3,550443808606126081,2015-01-01 01:10:05,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,apple
4,550443809700851716,2015-01-01 01:10:05,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,apple


In [214]:
def clean_body(col):
    import re
    # usun urle
    # usun hashtagi 
    pattern_url = r'https?://[a-z.]+/[?a-z0-9./]+'
    pattern_hash = r'#[a-z0-9_]+'
    col = re.sub(pattern_url, '', col, flags=re.I)
    col = re.sub(pattern_hash, '', col, flags=re.I)

    return col

In [215]:
xd = tweets['body'][37]
clean_body(xd)

'Facebook, Apple, And Microsoft ganging up on Google - Business Insider  $FB, $AAPL, $GOOG, $MSFT '

In [216]:
tweets_clean = tweets['body'].apply(clean_body)

In [217]:
tweets_sent = tweets_clean.apply(analyzer.polarity_scores)

In [218]:
tweets_sent[:5]

0    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1    {'neg': 0.339, 'neu': 0.661, 'pos': 0.0, 'comp...
2    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
3    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
4    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
Name: body, dtype: object

In [219]:
def convert_dict_keyval_to_col(dict, key):
    new_col = []
    for i in dict:
        new_col.append(i[key])
    return new_col

In [220]:
sentiment_val = pd.DataFrame()
sentiment_val['pos'] = convert_dict_keyval_to_col(tweets_sent, 'pos')
sentiment_val['neu'] = convert_dict_keyval_to_col(tweets_sent, 'neu')
sentiment_val['neg'] = convert_dict_keyval_to_col(tweets_sent, 'neg')
sentiment_val['comp'] = convert_dict_keyval_to_col(tweets_sent, 'compound')


In [221]:
sentiment_val['date'] = tweets['post_date']
sentiment_val['ticker_symbol'] = tweets['ticker_symbol']
sentiment_val.head()

,pos,neu,neg,comp,date,ticker_symbol
0,0.0,1.000,0.000,0.0000,2015-01-01 01:00:57,AAPL
1,0.0,0.661,0.339,-0.8271,2015-01-01 01:01:36,AAPL
2,0.0,1.000,0.000,0.0000,2015-01-01 01:10:05,AAPL
3,0.0,1.000,0.000,0.0000,2015-01-01 01:10:05,AAPL
4,0.0,1.000,0.000,0.0000,2015-01-01 01:10:05,AAPL


### Aggregation

In [222]:
# round down
# problem moze sie pojawic w zaleznosci od tickow stokow
# (jaki?) 
def prepare_time(col):
    date, time = col.split(' ')
    time = f'{time[0:2]}:00:00'
    return date + ' ' + time

In [223]:
sentiment_val['date'] = sentiment_val['date'].apply(prepare_time)
sentiment_val['date'] = pd.to_datetime(sentiment_val['date'])
sentiment_val.drop(['ticker_symbol'], axis=1, inplace=True)
sentiment_val.head()

,pos,neu,neg,comp,date
0,0.0,1.000,0.000,0.0000,2015-01-01 01:00:00
1,0.0,0.661,0.339,-0.8271,2015-01-01 01:00:00
2,0.0,1.000,0.000,0.0000,2015-01-01 01:00:00
3,0.0,1.000,0.000,0.0000,2015-01-01 01:00:00
4,0.0,1.000,0.000,0.0000,2015-01-01 01:00:00


In [224]:
# grupuje godzinowo
#values=['comp','neg','neu','pos']
test = pd.pivot_table(sentiment_val, index=['date'], aggfunc=['max', 'min', 'std', 'mean', 'median', 'count']).reset_index()

In [225]:
test.columns = [f'{i}_{j}' for i,j in test.columns]
test['count'] = test['count_pos']

In [226]:
test.drop(['count_neg','count_neu','count_pos', 'count_comp'], axis=1, inplace=True)

test.head()

,date_,max_comp,max_neg,max_neu,max_pos,min_comp,min_neg,min_neu,min_pos,std_comp,...,std_pos,mean_comp,mean_neg,mean_neu,mean_pos,median_comp,median_neg,median_neu,median_pos,count
0,2015-01-01 01:00:00,0.6369,0.339,1.0,0.369,-0.8271,0.0,0.631,0.0,0.272298,...,0.096070,0.062482,0.023045,0.917818,0.059136,0.0000,0.000,1.0000,0.0,22
1,2015-01-01 02:00:00,0.7456,0.223,1.0,0.413,-0.5766,0.0,0.587,0.0,0.375012,...,0.124790,0.053925,0.053167,0.866500,0.080333,0.0000,0.000,0.8825,0.0,12
2,2015-01-01 03:00:00,0.8065,0.152,1.0,0.327,-0.3612,0.0,0.673,0.0,0.317696,...,0.117028,0.113570,0.015200,0.916700,0.068100,0.0000,0.000,1.0000,0.0,10
3,2015-01-01 04:00:00,0.4019,0.184,1.0,0.184,-0.2732,0.0,0.649,0.0,0.243724,...,0.086003,-0.070700,0.080000,0.869857,0.050286,-0.0772,0.123,0.8770,0.0,7
4,2015-01-01 05:00:00,0.5023,0.111,1.0,0.136,-0.2500,0.0,0.864,0.0,0.260410,...,0.062039,0.073563,0.023875,0.942625,0.033500,0.0000,0.000,0.9600,0.0,8


In [227]:
test = test.set_index('date_')
test_to_merge = test.between_time('00:00', '15:00')
test_to_merge.head()

,max_comp,max_neg,max_neu,max_pos,min_comp,min_neg,min_neu,min_pos,std_comp,std_neg,...,std_pos,mean_comp,mean_neg,mean_neu,mean_pos,median_comp,median_neg,median_neu,median_pos,count
date_,,,,,,,,,,,,,,,,,,,,,
2015-01-01 01:00:00,0.6369,0.339,1.0,0.369,-0.8271,0.0,0.631,0.0,0.272298,0.079120,...,0.096070,0.062482,0.023045,0.917818,0.059136,0.0000,0.000,1.0000,0.0,22
2015-01-01 02:00:00,0.7456,0.223,1.0,0.413,-0.5766,0.0,0.587,0.0,0.375012,0.083168,...,0.124790,0.053925,0.053167,0.866500,0.080333,0.0000,0.000,0.8825,0.0,12
2015-01-01 03:00:00,0.8065,0.152,1.0,0.327,-0.3612,0.0,0.673,0.0,0.317696,0.048067,...,0.117028,0.113570,0.015200,0.916700,0.068100,0.0000,0.000,1.0000,0.0,10
2015-01-01 04:00:00,0.4019,0.184,1.0,0.184,-0.2732,0.0,0.649,0.0,0.243724,0.077690,...,0.086003,-0.070700,0.080000,0.869857,0.050286,-0.0772,0.123,0.8770,0.0,7
2015-01-01 05:00:00,0.5023,0.111,1.0,0.136,-0.2500,0.0,0.864,0.0,0.260410,0.044978,...,0.062039,0.073563,0.023875,0.942625,0.033500,0.0000,0.000,0.9600,0.0,8


In [228]:
test_to_merge = test_to_merge.groupby(pd.Grouper(freq='D'))
merged = test_to_merge.mean()
merged['count'] = test_to_merge['count'].sum()
merged.head()

,max_comp,max_neg,max_neu,max_pos,min_comp,min_neg,min_neu,min_pos,std_comp,std_neg,...,std_pos,mean_comp,mean_neg,mean_neu,mean_pos,median_comp,median_neg,median_neu,median_pos,count
date_,,,,,,,,,,,,,,,,,,,,,
2015-01-01,0.652000,0.173200,0.959733,0.292667,-0.292373,0.005667,0.683733,0.017467,0.334046,0.062600,...,0.114573,0.184597,0.039461,0.847403,0.113155,0.147253,0.0146,0.859900,0.078667,129
2015-01-02,0.545244,0.188625,1.000000,0.314937,-0.371206,0.000000,0.677375,0.000000,0.262823,0.059053,...,0.101051,0.087522,0.024859,0.913108,0.062030,0.000000,0.0000,0.958281,0.000000,319
2015-01-03,0.688193,0.183600,0.968400,0.320467,-0.208740,0.000000,0.662267,0.031600,0.275100,0.048425,...,0.101789,0.303648,0.016291,0.849964,0.133745,0.244640,0.0000,0.849967,0.105633,214
2015-01-04,0.733463,0.182688,0.974500,0.346688,-0.252575,0.000000,0.628625,0.025500,0.330971,0.055049,...,0.116065,0.288477,0.023563,0.841769,0.134662,0.310609,0.0000,0.831625,0.117375,217
2015-01-05,0.850100,0.198000,1.000000,0.399000,-0.408550,0.000000,0.590437,0.000000,0.371980,0.054890,...,0.134793,0.254062,0.023951,0.846544,0.129516,0.235597,0.0000,0.852406,0.104313,393


In [229]:
merged = merged.reset_index()
merged.date_ = merged['date_'] +  pd.to_timedelta(1, unit='H')
merged = merged.set_index('date_')
merged.head()

,max_comp,max_neg,max_neu,max_pos,min_comp,min_neg,min_neu,min_pos,std_comp,std_neg,...,std_pos,mean_comp,mean_neg,mean_neu,mean_pos,median_comp,median_neg,median_neu,median_pos,count
date_,,,,,,,,,,,,,,,,,,,,,
2015-01-01 01:00:00,0.652000,0.173200,0.959733,0.292667,-0.292373,0.005667,0.683733,0.017467,0.334046,0.062600,...,0.114573,0.184597,0.039461,0.847403,0.113155,0.147253,0.0146,0.859900,0.078667,129
2015-01-02 01:00:00,0.545244,0.188625,1.000000,0.314937,-0.371206,0.000000,0.677375,0.000000,0.262823,0.059053,...,0.101051,0.087522,0.024859,0.913108,0.062030,0.000000,0.0000,0.958281,0.000000,319
2015-01-03 01:00:00,0.688193,0.183600,0.968400,0.320467,-0.208740,0.000000,0.662267,0.031600,0.275100,0.048425,...,0.101789,0.303648,0.016291,0.849964,0.133745,0.244640,0.0000,0.849967,0.105633,214
2015-01-04 01:00:00,0.733463,0.182688,0.974500,0.346688,-0.252575,0.000000,0.628625,0.025500,0.330971,0.055049,...,0.116065,0.288477,0.023563,0.841769,0.134662,0.310609,0.0000,0.831625,0.117375,217
2015-01-05 01:00:00,0.850100,0.198000,1.000000,0.399000,-0.408550,0.000000,0.590437,0.000000,0.371980,0.054890,...,0.134793,0.254062,0.023951,0.846544,0.129516,0.235597,0.0000,0.852406,0.104313,393


In [230]:
# drop merged
print(len(test))
test = test.drop(index=test.between_time('00:00', '15:00').index)
print(len(test))

41582
14264


In [231]:
connected = pd.concat([test, merged], axis=0)
len(connected)

16091

In [232]:
connected = connected.sort_index()
connected.head(10)

,max_comp,max_neg,max_neu,max_pos,min_comp,min_neg,min_neu,min_pos,std_comp,std_neg,...,std_pos,mean_comp,mean_neg,mean_neu,mean_pos,median_comp,median_neg,median_neu,median_pos,count
date_,,,,,,,,,,,,,,,,,,,,,
2015-01-01 01:00:00,0.652000,0.173200,0.959733,0.292667,-0.292373,0.005667,0.683733,0.017467,0.334046,0.062600,...,0.114573,0.184597,0.039461,0.847403,0.113155,0.147253,0.0146,0.859900,0.078667,129
2015-01-01 16:00:00,0.882900,0.197000,1.000000,0.442000,-0.401900,0.000000,0.558000,0.000000,0.345082,0.051905,...,0.135023,0.196769,0.021655,0.882655,0.095655,0.000000,0.0000,0.900000,0.000000,29
2015-01-01 17:00:00,0.888500,0.216000,1.000000,0.439000,-0.510600,0.000000,0.561000,0.000000,0.369050,0.059908,...,0.169676,0.223577,0.016615,0.836692,0.146692,0.077200,0.0000,0.843000,0.115000,13
2015-01-01 18:00:00,0.636900,0.088000,1.000000,0.292000,-0.102700,0.000000,0.708000,0.000000,0.256040,0.022722,...,0.111633,0.158687,0.005867,0.915400,0.078733,0.000000,0.0000,1.000000,0.000000,15
2015-01-01 19:00:00,0.577500,0.193000,1.000000,0.266000,-0.476700,0.000000,0.734000,0.000000,0.362548,0.075260,...,0.100669,0.122892,0.039154,0.878231,0.082615,0.000000,0.0000,0.861000,0.000000,13
2015-01-01 20:00:00,0.554200,0.168000,1.000000,0.326000,-0.422600,0.000000,0.674000,0.000000,0.311547,0.061290,...,0.126950,0.145041,0.031588,0.860412,0.108000,0.000000,0.0000,0.868000,0.076000,17
2015-01-01 21:00:00,0.817300,0.326000,1.000000,0.395000,-0.700300,0.000000,0.605000,0.000000,0.357496,0.110521,...,0.103115,-0.046781,0.050762,0.907476,0.041762,0.000000,0.0000,1.000000,0.000000,21
2015-01-01 22:00:00,0.757900,0.221000,1.000000,0.263000,-0.373600,0.000000,0.637000,0.000000,0.307741,0.085175,...,0.097867,0.093046,0.050308,0.878000,0.071692,0.000000,0.0000,1.000000,0.000000,13
2015-01-01 23:00:00,0.658800,0.366000,1.000000,0.474000,-0.636900,0.000000,0.526000,0.000000,0.274036,0.074109,...,0.108526,0.165632,0.018607,0.891857,0.089536,0.202300,0.0000,0.881000,0.101000,28


In [233]:
apple_stocks = pd.read_csv('stocks_clean/AAPL.csv')
apple_stocks = apple_stocks.set_index('date_')

In [235]:
print(len(apple_stocks), len(connected))

10569 16091


In [238]:
stocks_sentiment = pd.merge(conn)
stocks_sentiment.head(20)

,max_comp,max_neg,max_neu,max_pos,min_comp,min_neg,min_neu,min_pos,std_comp,std_neg,...,median_neu,median_pos,count,date_,ticker,open,high,low,close,vol
date_,,,,,,,,,,,,,,,,,,,,,
2015-01-01 01:00:00,0.652000,0.173200,0.959733,0.292667,-0.292373,0.005667,0.683733,0.017467,0.334046,0.062600,...,0.859900,0.078667,129,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 16:00:00,0.882900,0.197000,1.000000,0.442000,-0.401900,0.000000,0.558000,0.000000,0.345082,0.051905,...,0.900000,0.000000,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 17:00:00,0.888500,0.216000,1.000000,0.439000,-0.510600,0.000000,0.561000,0.000000,0.369050,0.059908,...,0.843000,0.115000,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 18:00:00,0.636900,0.088000,1.000000,0.292000,-0.102700,0.000000,0.708000,0.000000,0.256040,0.022722,...,1.000000,0.000000,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 19:00:00,0.577500,0.193000,1.000000,0.266000,-0.476700,0.000000,0.734000,0.000000,0.362548,0.075260,...,0.861000,0.000000,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 20:00:00,0.554200,0.168000,1.000000,0.326000,-0.422600,0.000000,0.674000,0.000000,0.311547,0.061290,...,0.868000,0.076000,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 21:00:00,0.817300,0.326000,1.000000,0.395000,-0.700300,0.000000,0.605000,0.000000,0.357496,0.110521,...,1.000000,0.000000,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 22:00:00,0.757900,0.221000,1.000000,0.263000,-0.373600,0.000000,0.637000,0.000000,0.307741,0.085175,...,1.000000,0.000000,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 23:00:00,0.658800,0.366000,1.000000,0.474000,-0.636900,0.000000,0.526000,0.000000,0.274036,0.074109,...,0.881000,0.101000,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
